In [145]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

In [181]:
yazio = pd.read_csv("csv/yazio.csv", index_col="name")

In [183]:
units = [[re.findall("[a-z]+", v)[0] for v in yazio[c].values if pd.notna(v)][0] for c in tqdm(yazio.columns)]

In [194]:
for c in yazio.columns:
    for i in yazio.index:
        v = yazio[c][i]
        if pd.notna(v) and isinstance(v, str):
            if "kcal" in v:
                yazio[c][i] = float(v.split("kcal")[0])

In [186]:
units = [u.replace("mg", "g") for u in units]

In [187]:
new_columns = ["{c} | {unit}".format(c=c, unit=u) for c, u in zip(*[yazio.columns, units])]

In [190]:
yazio.columns = new_columns